### multivariate linear regression
$$ y = w_0 + w_1  x_1 + w_2 x_2 + w_3  x_3 $$
compute $w_0,w_1,w_2,w_3$

In [1]:
!ls

README.md         data.csv          multi-linrg.ipynb multi-linrg.py


In [2]:
!head data.csv

x1,x2,x3,y
0.511942218869,0.412118485242,0.540302305868,3.13513359407
-0.385495035898,-0.756802495308,0.753902254343,1.18965565737
-0.0166378209851,0.989358246623,0.540302305868,2.4040293201
0.786973424983,0.14112000806,1.0,6.35996967803
0.395975554823,0.656986598719,0.540302305868,2.8825578735
1.41801127326,-0.279415498199,0.96017028665,5.96444747979
2.41666215151,0.0,-0.653643620864,8.67681330186
0.0107047943288,0.656986598719,-0.653643620864,3.80298438425
0.356687666195,-0.279415498199,-0.911130261885,2.43446475337


In [3]:
import numpy as np
raw = np.genfromtxt ('data.csv', delimiter=",",skip_header=1)

In [4]:
raw.shape

(1000, 4)

In [5]:
raw.dtype

dtype('float64')

In [6]:
y_raw=raw[:,-1] # y: last column

In [7]:
y_raw.shape

(1000,)

In [8]:
# x_0 = np.ones(1000), intercept term w0
# add x_0 as first column to construct X matrix
x_matrix = np.append(np.ones(1000).reshape(1000,1),raw[:,0:3],axis=1)

In [9]:
x_matrix.shape

(1000, 4)

### 1. OLS矩阵法

$\hat w=(X^TX)^{-1}X^Ty $

In [10]:
from numpy.linalg import inv

w = inv(np.dot(x_matrix.T,x_matrix)).dot(x_matrix.T).dot(y_raw)
w

array([ 2.03076198,  2.97396653, -0.54139002,  0.97132913])

so $w_0 = 2.03076198, w_1 = 2.97396653, w_2 = -0.54139002, w_3 = 0.97132913$

### 2. 随机梯度下降 stochastic gradient descent

* initial guess: w0,w1,w2,w3
* cost function:
$$ J(w) = \frac{1}{2} \sum_{i=1}^{1000} (w_0+w_1 x_1^{(i)}+w_2 x_2^{(i)}+w_3 x_3^{(i)} - y^{(i)})^2$$

* minimize cost function by update w using gradient descent algorithm
$$ w_j = w_j + \alpha \frac{\partial}{\partial w_j} J(w) $$
$$     = w_j + \alpha (y^{(i)})-\hat y^{(i)}))*x^{(i)} $$
* sgd algorithm:

`
loop :
    for i=1 to 1000:
        w_vector = w_vector + alpha*(y_raw - predicted_y)[i-th]*x[i-th_example]
`


In [11]:
def sgd1(ws,alpha,x,y):
    
    for j in xrange(100):
        for i in xrange(1000):
            pred_y = np.dot(x,ws)
            error = y - pred_y
            ws = ws + alpha*error[i]*x[i,:]
    return ws

In [12]:
ws = np.ones(4)
alpha = 0.001

sgd1(ws,alpha,x_matrix,y_raw)

array([ 2.06136271,  2.98939891, -0.53134609,  0.98906994])

发现和 matrix 方法得到的略有差别, 重新考虑了一下update w 的目的是使 cost function 最小.  
所以在每步更新 w 后比较 cost function 是否变小, 若没有变小, 就不更新这一步.

In [13]:
def cost_func(ws,x,y): #cost function J(w)
    return sum((np.dot(x,ws)-y)**2)/2

In [14]:
def sgd2(ws,alpha,x,y):
    
    for j in xrange(100):
        for i in xrange(1000):
            old_cost=cost_func(ws,x,y)
            pred_y = np.dot(x,ws)
            error = y - pred_y
            ws = ws + alpha*error[i]*x[i,:]
            new_cost=cost_func(ws,x,y)
            
            if (new_cost > old_cost):
                ws = ws - alpha*error[i]*x[i,:]

    return ws

In [15]:
ws = np.ones(4)
alpha = 0.001

sgd2(ws,alpha,x_matrix,y_raw)

array([ 2.03076186,  2.97397058, -0.54138078,  0.97133892])

In [16]:
%run multi-linrg.py

1st method - analytical solution by matrix
w0 = 2.030762
w1 = 2.973967
w2 = -0.541390
w3 = 0.971329
2nd method - stochastic gradient descent
w0 = 2.030762
w1 = 2.973971
w2 = -0.541381
w3 = 0.971339
